# EcoDRR — Master GeoAI Resilience Model (v1)
Full end-to-end notebook: GEE preproc, patch extraction, U-Net training & inference, DEM/hydrology, NDVI Δ, habitat quality, and DRR risk mapping. Use in Colab with GEE auth.

In [ ]:
!pip install rasterio

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.enums import Resampling
import torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
print('Core libs loaded')

Core libs loaded


## 1. GEE — authenticate & fetch
This section shows how to fetch Sentinel-2 composites, CHIRPS and NASADEM from Earth Engine. Run in Colab after ee.Authenticate() and ee.Initialize().

In [3]:
# Earth Engine block
try:
    import ee, geemap
    print('Earth Engine available')
except Exception as e:
    print('GEE not available:', e)


Earth Engine available


## 2. Local TIFF loader and preprocessing
Load `/content/data/sent_rgb.tif` and `/content/data/sent_ndvi.tif`, downsample and crop for patching.

In [7]:
import os
def load_downsample(path, max_dim=256, bands=None):
    with rasterio.open(path) as src:
        if bands is None: bands = src.indexes
        w,h = src.width, src.height
        scale = max(1, max(w,h)/max_dim)
        out_w,out_h = int(w/scale), int(h/scale)
        data = src.read(bands, out_shape=(len(bands),out_h,out_w), resampling=Resampling.bilinear)
    return data

def crop_center(data, win=128):
    _,H,W = data.shape
    cy,cx = H//2, W//2
    return data[:, cy-win//2:cy+win//2, cx-win//2:cx+win//2]

rgb_path='/content/data/sent_rgb.tif'
ndvi_path='/content/data/sent_ndvi.tif'
if os.path.exists(rgb_path) and os.path.exists(ndvi_path):
    rgb = load_downsample(rgb_path,256,[1,2,3])
    ndv = load_downsample(ndvi_path,256,[1])
    rgb128 = crop_center(rgb,128)
    ndv128 = crop_center(ndv,128)
    combined = np.concatenate([rgb128.astype('float32'), ndv128.astype('float32')], axis=0)
    print('Prepared combined array', combined.shape)
else:
    print('Missing sent_rgb.tif or sent_ndvi.tif in /content/data — please export from GEE or upload')


Prepared combined array (4, 128, 128)


## 3. Patch extraction, U-Net training & inference (light)

In [10]:
def extract_patches(image, ps=32):
    b,H,W = image.shape
    patches=[]
    for y in range(0,H-ps+1,ps):
        for x in range(0,W-ps+1,ps):
            patches.append(image[:, y:y+ps, x:x+ps])
    return np.stack(patches,0)

if 'combined' in globals():
    patches = extract_patches(combined,32)
    np.savez_compressed('/content/data/patches_rgb_ndvi.npz', patches=patches)
    print('Patches saved', patches.shape)

# Tiny UNet definition
class TinyUNet(nn.Module):
    def __init__(self, inc, cls):
        super().__init__()
        self.e1=nn.Sequential(nn.Conv2d(inc,16,3,padding=1),nn.ReLU(), nn.Conv2d(16,16,3,padding=1), nn.ReLU())
        self.p=nn.MaxPool2d(2)
        self.e2=nn.Sequential(nn.Conv2d(16,32,3,padding=1),nn.ReLU(), nn.Conv2d(32,32,3,padding=1), nn.ReLU())
        self.u=nn.ConvTranspose2d(32,16,2,stride=2)
        self.d1=nn.Sequential(nn.Conv2d(32,16,3,padding=1),nn.ReLU(), nn.Conv2d(16,16,3,padding=1), nn.ReLU())
        self.out=nn.Conv2d(16,cls,1)
    def forward(self,x):
        e1=self.e1(x); e2=self.e2(self.p(e1)); u=self.u(e2); d=self.d1(torch.cat([u,e1],1)); return self.out(d)

# Lightweight training if patches exist
if os.path.exists('/content/data/patches_rgb_ndvi.npz'):
    data = np.load('/content/data/patches_rgb_ndvi.npz')['patches']
    X = data.astype('float32'); X = X/(X.max()+1e-9)
    ndv = X[:,3:4]; bright = X[:,:3].mean(1)
    labels = np.zeros((X.shape[0],32,32), dtype=np.int64)
    labels[(ndv[:,0]>0.25)] = 1
    labels[(ndv[:,0]<=0.25)&(bright>0.4)] = 2
    labels[(ndv[:,0]<=0.25)&(bright<=0.4)] = 3
    ds = TensorDataset(torch.tensor(X), torch.tensor(labels))
    dl = DataLoader(ds, batch_size=4, shuffle=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TinyUNet(4,4).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()
    for ep in range(3):
        tot=0
        for xb,yb in dl:
            xb,yb=xb.to(device),yb.to(device)
            opt.zero_grad(); out=model(xb); loss=loss_fn(out,yb); loss.backward(); opt.step(); tot+=loss.item()
        print('Ep', ep+1, 'loss', tot/len(dl))
    os.makedirs('/content/data/models', exist_ok=True)
    torch.save(model.state_dict(), '/content/data/models/tiny_unet_trained.pth')
    print('Model trained and saved')
else:
    print('No patches found for training — run preprocessing or upload patches')


Patches saved (16, 4, 32, 32)
Ep 1 loss 1.2979334592819214
Ep 2 loss 1.281512200832367
Ep 3 loss 1.2609952986240387
Model trained and saved


End of Master notebook; use specific sections to run full experiments.